<a href="https://colab.research.google.com/github/justinline/coastergan/blob/master/coaster_gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@markdown #Project settings
#@markdown .csv file with single column of values as floats i.e 

#@markdown `0.1`

#@markdown `-0.3`

#@markdown `0`
psi_csv_file = '/content/drive/My Drive/test/psi.csv' #@param {type:"string"}

#@markdown Use the psi map?
use_psi = True #@param {type:"boolean"}

#@markdown Space separated values in the format: `[1 2 3][2 3 4][3 4 5]` (.txt file)
vector_txt_file = '/content/drive/My Drive/test/input.txt' #@param {type:"string"}
#@markdown This uses the directory `/content/drive/My Drive/test/output` for outputting images and video
#@markdown model is loaded at  `/content/drive/My Drive/test/model.pkl`


In [ ]:
import csv

psi = []

with open(psi_csv_file, newline='') as csvfile:
     reader = csv.reader(csvfile, delimiter=' ')
     for row in reader:
        # print(row[1][:-1])
        # print(float(row[1][:-1]))
        psi.append(round(float(row[0]), 3))

# print(psi)

In [ ]:
import re


# output_file = '/content/drive/My Drive/test/output.txt'
with open(vector_txt_file, 'r') as file:
  uploaded_raw_string = file.read()

# print(uploaded_raw_string)

# Replaces the space separated values with commas, [1 2 3 4][1 2 3 4] => [[1, 2, 3, 4, ], [1, 2, 3, 4, ]]
# and evaluates it as a python list
comma_separated = re.sub(r"(\d) ", "\g<1>,\n", uploaded_raw_string)
comma_separated_arrays = re.sub(r"(\d])", "\g<1>,", comma_separated)
# 2. => 2.0
missing_post_decimal = re.sub(r"(\.)\s", "\g<1>0, ", comma_separated_arrays)
enclosed = f'[{missing_post_decimal}]'
# debugging
# f = open(output_file, "w")
# f.write(enclosed)
# f.close()
raw_eval = eval(enclosed)


In [ ]:
# Check the GPU
!nvidia-smi

Tue Feb  1 15:42:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    43W / 250W |   8223MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Clone the repo
!git clone https://github.com/justinline/stylegan2-ada
!pip install -e ./stylegan2-ada


fatal: destination path 'stylegan2-ada' already exists and is not an empty directory.
Obtaining file:///content/drive/MyDrive/test/output/stylegan2-ada
  Attempting uninstall: hiddenvortices
    Found existing installation: hiddenvortices 1.0
    Can't uninstall 'hiddenvortices'. No files were found to uninstall.
  Running setup.py develop for hiddenvortices


In [ ]:
# !python ./stylegan2-ada/generate.py generate-images --network="/content/drive/My Drive/test/model.pkl" --seeds=0-10 --outdir="/content/drive/My Drive/test/output"
%tensorflow_version 1.x

In [ ]:
import sys
import os
# Mount our stylegan repo as a module
sys.path.append('./stylegan2-ada')

# Remove google drive images + video from output folder
path = "/content/drive/My Drive/test/output"
imgs = os.listdir(path)
for img in imgs:
    if (os.path.isdir(img)): continue
    os.remove(f'{path}/{img}')

In [ ]:
from google.colab.patches import cv2_imshow
import cv2
import dnnlib.tflib as tflib
import pickle
import os
import dnnlib
import numpy as np
import random

input = np.array(raw_eval)

#####
# Generates coaster images based on a trained pickle model file location
# in GDrive as a parameter
#####
def generate_images(network_pkl):
    # TF stuff
    tflib.init_tf()
    print('Loading networks from "%s"...' % network_pkl)
    with dnnlib.util.open_url(network_pkl) as fp:
        _G, _D, Gs = pickle.load(fp)

    Gs_kwargs = {
        'output_transform': dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True),
        'randomize_noise': False
    }

    # Initial styleGAN parameters
    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
    label = np.zeros([1] + Gs.input_shapes[1][1:])

    # def getZOSC(*args):
    #     global z
    #     z = np.array(args).reshape(1,512)

    #@markdown #Generation settings
    show_output_image = False #@param {type:"boolean"}
    zeros = 5     # TODO: This is a quick hacky solution to filenames
    Gs_kwargs['truncation_psi'] = 1
    # Begin the generation and output images
    os.chdir("/content/drive/My Drive/test/output/")
    component = Gs.input_shape[1:] # [512]
    noise_rnd = np.random # fix noise
    tflib.set_vars({var: noise_rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]
    label = np.zeros([1] + Gs.input_shapes[1][1:])
    
    for index, value in enumerate(input):
      z = input[index].reshape(1,512)
      if use_psi:
        Gs_kwargs['truncation_psi'] = psi[index]
        print('psi:', Gs_kwargs['truncation_psi'])

      image = Gs.run(z, label, **Gs_kwargs)[0]

      file_number = str(index).zfill(zeros)
      # Write the image  to g drive and show it.
      cv2.imwrite("output" + file_number + ".jpg", image)
      print("Wrote: output" + file_number + ".jpg")
      if show_output_image is True: cv2_imshow(image)


# Let's run this function!
generate_images("/content/drive/My Drive/test/model.pkl")

Streaming output truncated to the last 5000 lines.
psi: 0.868
Wrote: output01627.jpg
psi: 0.869
Wrote: output01628.jpg
psi: 0.87
Wrote: output01629.jpg
psi: 0.871
Wrote: output01630.jpg
psi: 0.871
Wrote: output01631.jpg
psi: 0.872
Wrote: output01632.jpg
psi: 0.873
Wrote: output01633.jpg
psi: 0.874
Wrote: output01634.jpg
psi: 0.875
Wrote: output01635.jpg
psi: 0.875
Wrote: output01636.jpg
psi: 0.876
Wrote: output01637.jpg
psi: 0.877
Wrote: output01638.jpg
psi: 0.878
Wrote: output01639.jpg
psi: 0.878
Wrote: output01640.jpg
psi: 0.88
Wrote: output01641.jpg
psi: 0.88
Wrote: output01642.jpg
psi: 0.881
Wrote: output01643.jpg
psi: 0.882
Wrote: output01644.jpg
psi: 0.883
Wrote: output01645.jpg
psi: 0.884
Wrote: output01646.jpg
psi: 0.884
Wrote: output01647.jpg
psi: 0.886
Wrote: output01648.jpg
psi: 0.886
Wrote: output01649.jpg
psi: 0.887
Wrote: output01650.jpg
psi: 0.888
Wrote: output01651.jpg
psi: 0.889
Wrote: output01652.jpg
psi: 0.889
Wrote: output01653.jpg
psi: 0.89
Wrote: output01654.jpg
p

In [ ]:
# Convert image sequence to video
!cd "/content/drive/My Drive/test/output/"
# Files must be 01.jpg 02.jpg 03 etc if n<100 image or 001 002 if 100>n<1000 
!ffmpeg -y -pattern_type glob -i "*.jpg" -c:v libx264 -pix_fmt yuv420p -framerate 3 -crf 23 out.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [ ]:
# Reverse the output video
!ffmpeg -y -i "/content/drive/My Drive/test/output/out.mp4" -vf reverse out-reversed.mp4


ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib